# SPR 2026 - BERTimbau v4 (Threshold Tuning Apenas)

**Score baseline:** 0.79696 (CAMPEÃO!)

**Estratégia v4:** NÃO MEXER NO MODELO - apenas threshold tuning
- Usar modelo exatamente como está (config original)
- Ajustar APENAS thresholds na inferência
- Thresholds mais sensíveis para classes 5 e 6

**IMPORTANTE:** O modelo original funciona bem. Alterações quebraram v3.

**Meta:** Manter 0.79+ ou melhorar para 0.80+

---
## 📥 MODELO - Baixar com: `models/download_bertimbau_large.ipynb`

**Kaggle Models:** Add Input → Models → `bertimbau-ptbr-complete`

---
**CONFIGURAÇÃO KAGGLE:**
1. Settings → Internet → **OFF**
2. Settings → Accelerator → **GPU T4 x2**
---

In [ ]:
# ===== SPR 2026 - BERTIMBAU v4 (THRESHOLD TUNING APENAS) =====

print("[1/7] Configurando ambiente...")
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# ========== CONFIGURAÇÕES ORIGINAIS - NÃO ALTERAR ==========
SEED = 42
MAX_LEN = 256
BATCH_SIZE = 8
EPOCHS = 5
LR = 2e-5
NUM_CLASSES = 7
FOCAL_GAMMA = 2.0
FOCAL_ALPHA = 0.25
# ===========================================================

# ========== THRESHOLD TUNING - AJUSTAR AQUI ==========
USE_THRESHOLD_TUNING = True
THRESHOLDS = {
    0: 0.50,
    1: 0.50,
    2: 0.50,
    3: 0.50,
    4: 0.50,
    5: 0.30,   # Classe 5 - mais sensível
    6: 0.25,   # Classe 6 - muito mais sensível
}
# ====================================================

DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'

# ========== VERIFICAR DATASET PRIMEIRO ==========
if not os.path.exists(DATA_DIR):
    print("\n" + "="*60)
    print("ERRO: Dataset não encontrado!")
    print("="*60)
    print("\nAdicione o dataset:")
    print("Add Input → Competition → spr-2026-mammography-report-classification")
    raise FileNotFoundError(f"Dataset não encontrado: {DATA_DIR}")
print(f"Dataset: {DATA_DIR}")

# Auto-detectar modelo
def find_model_path():
    base = '/kaggle/input'
    def has_config(path):
        return os.path.isdir(path) and os.path.exists(os.path.join(path, 'config.json'))
    def search_dir(directory, depth=0, max_depth=10):
        if depth > max_depth:
            return None
        try:
            for item in os.listdir(directory):
                path = os.path.join(directory, item)
                if os.path.isdir(path):
                    if has_config(path):
                        return path
                    result = search_dir(path, depth + 1, max_depth)
                    if result:
                        return result
        except:
            pass
        return None
    return search_dir(base)

MODEL_PATH = find_model_path()
if MODEL_PATH is None:
    raise FileNotFoundError("Adicione o modelo BERTimbau ao notebook!")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
print(f'Model: {MODEL_PATH}')
torch.manual_seed(SEED)
np.random.seed(SEED)

In [ ]:
# Focal Loss - CONFIGURAÇÃO ORIGINAL
print("[2/7] Definindo Focal Loss (config original)...")

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
    
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        return focal_loss

criterion = FocalLoss(alpha=FOCAL_ALPHA, gamma=FOCAL_GAMMA)
print(f'Focal Loss: gamma={FOCAL_GAMMA}, alpha={FOCAL_ALPHA}')

In [ ]:
# Carregar dados
print("[3/7] Carregando dados...")
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f'Train: {train_df.shape}, Test: {test_df.shape}')

# Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_len=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        enc = self.tokenizer(
            str(self.texts[idx]),
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt',
        )
        item = {
            'input_ids': enc['input_ids'].squeeze(),
            'attention_mask': enc['attention_mask'].squeeze(),
        }
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

In [ ]:
# Preparar dados
print("[4/7] Preparando datasets...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['report'].values, train_df['target'].values,
    test_size=0.1, random_state=SEED, stratify=train_df['target']
)

train_ds = TextDataset(train_texts, train_labels, tokenizer, MAX_LEN)
val_ds = TextDataset(val_texts, val_labels, tokenizer, MAX_LEN)
test_ds = TextDataset(test_df['report'].values, None, tokenizer, MAX_LEN)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

print(f'Train: {len(train_ds)}, Val: {len(val_ds)}, Test: {len(test_ds)}')

In [ ]:
# Carregar e treinar modelo
print("[5/7] Carregando e treinando modelo...")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH, num_labels=NUM_CLASSES, local_files_only=True
)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

best_f1 = 0
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS}'):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_loss += loss.item()
    
    # Validação
    model.eval()
    val_preds, val_true = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = outputs.logits.argmax(dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_true.extend(batch['labels'].numpy())
    
    val_f1 = f1_score(val_true, val_preds, average='macro')
    print(f'Epoch {epoch+1}: Loss={train_loss/len(train_loader):.4f}, Val F1={val_f1:.4f}')
    
    if val_f1 > best_f1:
        best_f1 = val_f1

print(f'\nMelhor Val F1: {best_f1:.4f}')

In [ ]:
# Inferência com threshold tuning
print("[6/7] Inferência com threshold tuning...")

model.eval()
all_probs = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Inferência'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probs = F.softmax(outputs.logits, dim=1)
        all_probs.append(probs.cpu().numpy())

all_probs = np.vstack(all_probs)
print(f"Probabilidades shape: {all_probs.shape}")

# Aplicar threshold tuning
if USE_THRESHOLD_TUNING:
    print("\nAplicando threshold tuning...")
    print("Thresholds:")
    for c, t in THRESHOLDS.items():
        print(f"  Classe {c}: {t}")
    
    predictions = []
    for i in range(len(all_probs)):
        adj_probs = all_probs[i].copy()
        for c, t in THRESHOLDS.items():
            if c < len(adj_probs):
                adj_probs[c] *= (0.5 / t)
        predictions.append(np.argmax(adj_probs))
    predictions = np.array(predictions)
else:
    predictions = np.argmax(all_probs, axis=1)

In [ ]:
# Submissão
print("[7/7] Gerando submissão...")

sample_path = f'{DATA_DIR}/sample_submission.csv'
if os.path.exists(sample_path):
    sample_sub = pd.read_csv(sample_path)
    SUB_ID = sample_sub.columns[0]
    SUB_LABEL = sample_sub.columns[1]
else:
    SUB_ID = 'ID'
    SUB_LABEL = 'target'

submission = pd.DataFrame({SUB_ID: test_df['ID'], SUB_LABEL: predictions})
submission.to_csv('/kaggle/working/submission.csv', index=False)

print("="*60)
print("BERTimbau v4 CONCLUÍDO!")
print("="*60)
print(f"\nMelhor Val F1: {best_f1:.4f}")
print(f"Threshold tuning: {'ATIVO' if USE_THRESHOLD_TUNING else 'DESATIVADO'}")
print("\nDistribuição das predições:")
print(submission[SUB_LABEL].value_counts().sort_index())
print("\n✅ submission.csv criado!")